In [ ]:
from flask import Flask, render_template

app = Flask(__name__)

@app.route('/')
def home():
    # Sample data for demonstration
    user_profile = {
        'photo': 'path/to/photo.jpg',
        'name': 'John Doe',
        'company_name': 'ABC Corp',
        'designation': 'Software Engineer',
        'social_media': {
            'youtube': 'https://youtube.com/example',
            'instagram': 'https://instagram.com/example',
            'facebook': 'https://facebook.com/example',
        }
    }
    return render_template('index.html', user_profile=user_profile)

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model

# Load face detection model
face_net = cv2.dnn.readNetFromCaffe("face_detector/deploy.prototxt", "face_detector/res10_300x300_ssd_iter_140000.caffemodel")

# Load mask detection model
mask_net = load_model("mask_detector.model")

def detect_mask(frame):
    (h, w) = frame.shape[:2]

    # Construct a blob from the image
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), (104.0, 177.0, 123.0))

    # Pass the blob through the face detection model
    face_net.setInput(blob)
    detections = face_net.forward()

    # Loop over the detections
    for i in range(0, detections.shape[2]):
        confidence = detections[0, 0, i, 2]

        # Filter out weak detections
        if confidence > 0.5:
            # Compute the (x, y)-coordinates of the bounding box
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Extract the face ROI, resize it to 224x224, and preprocess
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = preprocess_input(face)
            face = np.expand_dims(face, axis=0)

            # Pass the face through the mask detection model
            (mask, without_mask) = mask_net.predict(face)[0]

            # Determine the class label and color
            label = "Mask" if mask > without_mask else "No Mask"
            color = (0, 255, 0) if label == "Mask" else (0, 0, 255)

            # Include the probability in the label
            label = "{}: {:.2f}%".format(label, max(mask, without_mask) * 100)

            # Display the label and bounding box rectangle on the output frame
            cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 2)
            cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    return frame

# Capture video from webcam (change 0 to the video file path if using a file)
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    # Perform mask detection on the frame
    output_frame = detect_mask(frame)

    # Display the output frame
    cv2.imshow("Face Mask Detection", output_frame)

    # Break the loop when 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()
